In this notebook, we will calculate features using Trotterization with a noisy classical simulator (AerSimulator)

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['x', 'reset', 'measure', 'cz', 'sx', 'id']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('x', (0,)), ('x', (1,)), ('x', (2,))

### Prepare parameters and load dataset

In [7]:
n_qubits = 52

C = 3
n_samples = 55
n_shots = 100

if n_qubits <= 50:  # n_qubits <= 50
    threshold = 0.6

    # Load dataset (orient="records")
    df_n_step = pd.read_json(
        f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
        orient="records",
    )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 50
    threshold = None
    n_features = 20
    max_time = 20
    # t_k = k * pi / C
    times = np.arange(0, max_time, np.pi / C)

    # n_qubits > 50 の場合は、n_step は 1 から 7 の整数値を取る。
    # まず 1から7までの20個の等間隔の数値を生成
    n_step_array = np.linspace(1, 7, 20)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 52
C: 3
threshold: None
n_samples: 55
n_shots: 100
n_features: 20
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306
 12.56637061 13.61356817 14.66076572 15.70796327 16.75516082 17.80235837
 18.84955592 19.89675347]
n_step_array: [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.021581075300000002, -0.004814892800000001, ...",0.874333,NaN,NaN
1,1,"[-0.0424831577, 0.024743239900000002, 0.014216...",1.264982,NaN,NaN
2,2,"[-0.019849499200000002, 0.039776682300000005, ...",1.043115,NaN,NaN
3,3,"[0.0038696408, -0.0280307067, 0.01233447190000...",0.878768,NaN,NaN
4,4,"[-0.038509265, -0.021418483500000002, -0.02916...",0.920255,NaN,NaN
5,5,"[0.0169817178, 0.021851908, -0.027246614500000...",1.089466,NaN,NaN
6,6,"[-0.0100976512, 0.0122639897, -0.0205218139, -...",0.917398,NaN,NaN
7,7,"[0.0228687258, -0.0005996629, 0.00761107880000...",0.887918,NaN,NaN
8,8,"[-0.024747151000000002, 0.0313821979, 0.010417...",1.136008,NaN,NaN
9,9,"[0.0145011702, 0.0076980293, -0.0377251604, 0....",1.025680,NaN,NaN


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    # "backend": backend_dm_noisy,  # This is important!!
    "backend": backend_mps_noisy,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [ ]:
%%time
# For simulator, we need job instances instead of job ids
_, jobs, _ = run_job(config, sim_type="noisy")

Preparing circuits for sample 0/55


Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Preparing circuits for sample 28/55
P

In [9]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

Post-processing sample 38/55
Post-processing sample 39/55
Post-processing sample 40/55
Post-processing sample 41/55
Post-processing sample 42/55
Post-processing sample 43/55
Post-processing sample 44/55
Post-processing sample 45/55
Post-processing sample 46/55
Post-processing sample 47/55
Post-processing sample 48/55
Post-processing sample 49/55
Post-processing sample 50/55
Post-processing sample 51/55
Post-processing sample 52/55
Post-processing sample 53/55
Post-processing sample 54/55


,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,0.937500,0.689587,-0.229130,0.469491,-0.368162,0.239895,-0.325551,0.094243,-0.163047,...,0.003858,0.059508,-0.077585,0.030263,-0.217255,-0.080562,-0.307849,-0.176784,-0.309825,0.792109
1,1,0.909180,0.755348,0.027183,0.667724,0.102088,0.535889,0.113561,0.343682,0.059963,...,-0.100874,-0.128379,-0.179023,-0.208711,-0.248033,-0.204621,-0.273028,-0.224936,-0.299190,1.102887
2,2,0.934570,0.773981,0.071705,0.714103,0.145320,0.610827,0.176676,0.467015,0.215232,...,0.110241,0.170245,0.106771,0.136615,0.035690,0.107147,0.067460,0.103760,0.065364,1.130663
3,3,0.919922,0.670222,0.245851,0.388215,0.339756,0.100387,0.256384,0.010913,0.039646,...,0.055958,0.041757,0.054545,0.121835,0.049488,0.159579,-0.005038,0.237930,0.077200,1.552866
4,4,0.922852,0.766576,0.100779,0.723552,0.160521,0.567209,0.230897,0.472385,0.269031,...,0.259081,0.085471,0.231618,-0.002232,0.123042,-0.082262,0.091590,-0.109548,0.062197,1.149036
5,5,0.913086,0.771073,0.207380,0.547459,0.325024,0.340856,0.341366,0.146112,0.237335,...,-0.096845,0.415100,-0.027654,0.339135,0.146516,0.278014,0.252555,0.132041,0.240608,1.332599
6,6,0.921875,0.686513,0.044433,0.466333,-0.003102,0.170926,-0.003787,-0.006125,0.022812,...,-0.180514,-0.185450,-0.175690,-0.210936,-0.050409,-0.103547,0.021334,-0.028725,0.016335,1.114135
7,7,0.910156,0.727865,-0.015345,0.553081,-0.041577,0.332008,-0.034057,0.074568,-0.024754,...,0.014798,0.032088,0.010350,-0.032437,-0.031215,-0.088108,-0.010192,-0.104024,0.026047,1.034106
8,8,0.919922,0.718727,0.226182,0.518480,0.352052,0.270094,0.330352,0.056931,0.248459,...,-0.015929,0.330963,0.055069,0.276631,0.085838,0.235371,0.133795,0.206065,0.207343,1.421492
9,9,0.917969,0.662896,-0.391488,0.405867,-0.628189,0.081692,-0.658197,-0.233871,-0.569205,...,0.065036,-0.282381,0.206357,-0.167096,0.219452,0.033685,0.241073,0.133769,0.183548,0.634733
